# 감성분석

> 나이브 베이즈 분류 모형을 이용하여 문서에 대한 감성분석
- 문서에 대한 positive 또는 negative 평가를 내리는것
- 샘플데이터로는 네이버 영화 감상평 사용
    - https://github.com/e9t/nsmc

In [ ]:
#%%time
## bash 에서 직접 해야함
#!rm -f ratings_train.txt ratings_test.txt
#!wget -nc https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
#!wget -nc https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

In [3]:
!ls |grep rating

ratings_test.txt
ratings_train.txt


In [14]:
### 유니코드 인코딩 하기 위해 codecs 사용
import codecs
with codecs.open("ratings_train.txt", encoding='utf-8') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]   # header 제외

- 내용은 X
- 평점은 y

In [11]:
from pprint import pprint
pprint(data[0])

['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']


In [21]:
X=list(zip(*data))[1] #데이터의 모든 줄(요소)의 1번째 요소를 모아 list
X[:3]

('아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다')

In [16]:
X = list(zip(*data))[1] #리뷰 본문
y = np.array(list(zip(*data))[2],dtype=int) #영화 평점

In [27]:
# 다항 나이브 베이즈 모형으로 학습
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

model1 = Pipeline([
    ('vect', CountVectorizer()),
    ('mb', MultinomialNB()),
])

In [28]:
model1.fit(X,y)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [29]:
import codecs
with codecs.open("ratings_test.txt", encoding='utf-8') as f:
    data_test = [line.split('\t') for line in f.read().splitlines()]
    data_test = data_test[1:]  
    
X_test = list(zip(*data_test))[1]
y_test = np.array(list(zip(*data_test))[2], dtype=int)

print(classification_report(y_test, model1.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83     24827
           1       0.84      0.81      0.82     25173

    accuracy                           0.83     50000
   macro avg       0.83      0.83      0.83     50000
weighted avg       0.83      0.83      0.83     50000



In [39]:
# TFIDF방법으로 했을때
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

X = list(zip(*data))[1] #리뷰 본문
y = np.array(list(zip(*data))[2],dtype=int) #영화 평점


model2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('mb', MultinomialNB()),
])



In [40]:
model2.fit(X, y)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [41]:
print(classification_report(y_test, model2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83     24827
           1       0.84      0.81      0.83     25173

    accuracy                           0.83     50000
   macro avg       0.83      0.83      0.83     50000
weighted avg       0.83      0.83      0.83     50000



### 형태소 분석기를 사용한 결과와 비교

In [10]:
import numpy as np

In [12]:
np.__path__

['C:\\ProgramData\\Anaconda3\\lib\\site-packages\\numpy']

In [1]:
import konlpy

In [2]:
from konlpy.tag import Kkma
pos_tagger = Kkma()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc)]

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
model3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_pos)),
    ('mb', MultinomialNB()),
])

In [4]:
model3.fit(X, y)

NameError: name 'X' is not defined

In [5]:
from konlpy.tag import *

hannanum = Hannanum()
kkma = Kkma()


ModuleNotFoundError: No module named 'konlpy.about'

In [1]:
from konlpy.corpus import kolaw
kolaw.fileids()
c = kolaw.open('constitution.txt').read()
print(c[:40])

ModuleNotFoundError: No module named 'konlpy.about'

In [45]:
kkma.morphs(c[:40])

TypeError: No matching overloads found for kr.lucypark.kkma.KkmaInterface.morphAnalyzer(str), options are:
	public java.util.List kr.lucypark.kkma.KkmaInterface.morphAnalyzer(java.lang.String) throws java.lang.Exception

	at JPMethod::findOverload(native\common\jp_method.cpp:242)
	at JPMethod::findOverload(native\common\jp_method.cpp:245)
	at JPMethod::invoke(native\common\jp_method.cpp:253)
	at PyJPMethod::__call__(native\python\pyjp_method.cpp:167)


'0.5.1'
https://github.com/konlpy/konlpy/issues/81?fbclid=IwAR0gHoR0F1u40Sb1jRRGIIqjVI3LCXkRC0-4brP3PvFY7CSXuaJKoNSx0Ro


https://konlpy-ko.readthedocs.io/ko/v0.4.3/install/

In [2]:
import konlpy
konlpy.__version__

ModuleNotFoundError: No module named 'konlpy.about'